#### Loading the Processed Controls and Treated data and calculate the Treatment Effects

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
import pandas as pd
import numpy as np

import time 
import datetime as dt

from pathlib import Path

import dowhy

ModuleNotFoundError: No module named 'dowhy'

In [ ]:
patht = '../../datap/treated.p'
pathc = '../../datap/control.p'

treated = pd.read_pickle(patht)
control = pd.read_pickle(pathc)

In [ ]:
control.shape, treated.shape

#### Get the Covariates for controls (First month will be Beginning, Last Month Outcome)

In [ ]:
def get_covariates( group ):
    """
    assumption, all ids have their months
    sorted
    
    """
    indexes = [0,-1]
    new_group = group.iloc[indexes]
    new_group['FF'] = 0
    new_group.iloc[-1, -1] = 1
    outcome = int((new_group['RESULT_VALUE'].iloc[0] - new_group['RESULT_VALUE'].iloc[-1]) > 0)
    new_group['OUTCOME'] = outcome
    
    return new_group

In [ ]:
%%time
controls2 = control.groupby(by = 'PERS_ID').apply(get_covariates)

In [ ]:
controls2.index = range(len(controls2))

#### GET THE OUTCOMES FOR TREATED

In [ ]:
treated.head()

In [ ]:
filterT = treated['FF'] == 0
filterC = controls2['FF'] == 0

In [ ]:
treated_F = treated[filterT]
control_F = controls2[filterC]

In [ ]:
treated_F.shape, control_F.shape

#### Keep only Necessary Columns For Matching (Treated)

In [ ]:
columns_to_drop_treated = ['PERS_ID', 'MYR','EVENT_DATE2', 'BIOMETRIC_CATEGORY', 'lengths', 'MYR_DATE', "REGISTER_DATE",
                   "LAST_ACTIVITY_DATE", "REGISTER_MYR", "LAST_ACTIVITY_MYR", "REF_START", "REF_END", "FF" ]

treated_cov = treated_F.drop(labels = columns_to_drop_treated, axis = 1).reset_index(drop = True)

In [ ]:
treated_cov

#### Keep Only Necessary Columns For Matching (Controls)

In [ ]:
columns_to_drop_controls = ['PERS_ID', 'MYR','EVENT_DATE2', 'BIOMETRIC_CATEGORY', 'lengths', 'MYR_DATE', "REGISTER_MYR", "LAST_ACTIVITY_MYR", "FF" ]

controls_cov = control_F.drop(labels = columns_to_drop_controls, axis = 1).reset_index(drop = True)

In [ ]:
all(controls_cov.columns == treated_cov.columns)

#### Make gender a categorical value

In [ ]:
treated_cov['MBR_GNDR'] = treated_cov["MBR_GNDR"].replace(['F','M'], [0,1])
controls_cov['MBR_GNDR'] = controls_cov["MBR_GNDR"].replace(['F','M'], [0,1])

In [ ]:
df = pd.concat((treated_cov, controls_cov), ignore_index = True, axis = 0)
common_causes = np.setdiff1d(df.columns, ['TREATED', 'OUTCOME'])

#### At this Point everything is Ready To Calculate Treatement Effects

#### Use doWhy (Some Problems)

In [ ]:
from dowhy import CausalModel
#import econml

filt = df.TREATED > 0
df.loc[filt, 'TREATED'] = True
df.loc[~filt, 'TREATED'] = False

model = CausalModel(df, outcome = 'OUTCOME', treatment = 'TREATED')#, common_causes = common_causes)
identified_estimand = model.identify_effect()
###errors

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.preprocessing import StandardScaler

#### Propensity Matching and ATE Estimation

In [ ]:
X, Y = df[common_causes].values, df.TREATED.values 
Xnorm  = StandardScaler().fit_transform(X)

In [ ]:
model = LR()
model = model.fit(Xnorm,Y)

In [ ]:
probas = model.predict_proba(Xnorm)

In [ ]:
treated = df.TREATED == 1
probas_t = probas[treated][:,1]
probas_c = probas[~treated][:,1]

In [ ]:
%%time
mini = []
argmin = []

for i in range(len(probas_t)):
    dist = np.sqrt((probas_c - probas_t[i])**2)
    mini.append(np.min(dist))
    argmin.append(np.argmin(dist))

#### With Matching

In [ ]:
treated_F.OUTCOME.mean() - control_F.iloc[argmin].OUTCOME.mean()

#### Without Matching

In [ ]:
treated_F.OUTCOME.mean() - control_F.OUTCOME.mean()

#### At this Point We can Save the Ids to Use for Survival Analysis

In [ ]:
ids_control = control_F.iloc[argmin].PERS_ID
ids_treated = treated_F.PERS_ID